In [1]:
!pip install gurobipy

You should consider upgrading via the 'C:\Users\artur\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# A TransShipment Problem

![slide1.png](https://i.ibb.co/XLWcpmr/download.png)


negative values are demand
positive values are supply
0 means a distribution center
Cij is the cost
Uij is the capacity to send product

In [2]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb

To define our node id's we use a python generator to generate a number for each node

In [3]:
N = [str(i+1) for i in range(6)]
N

['1', '2', '3', '4', '5', '6']

Now we provide the supply/demand in the sequence of our node ids.

In [4]:
supply = [5, 2, 0, -2, -4, -1]

Just to make sure we generated a label for each node and put in a supply for each node we include an assert.

In [5]:
assert len(supply) == len(N)

We can now convert this into a dictionary of values, with node IDs used as keys.

In [6]:
D = dict(zip(N, supply))
D

{'1': 5, '2': 2, '3': 0, '4': -2, '5': -4, '6': -1}

We give the cost of each link as a dictionary with a tuple (origin, destination). The cost from Node o to Node d can be retrieved with C\[(o,d)]

In [7]:
C = {('1', '2'): 1,
     ('1', '3'): 4,
     ('2', '3'): 2,
     ('3', '4'): 2,
     ('3', '5'): 5,
     ('3', '6'): 8,
     ('4', '5'): 1}

Similarily we list the capacity of each link.

In [8]:
CAPACITY = { ('1', '2'): 3,
             ('1', '3'): 3,
             ('2', '3'): 7,
             ('3', '4'): 5,
             ('3', '5'): 7,
             ('3', '6'): 1,
             ('4', '5'): 3}

In [9]:
#Just to check if we did not make an input error we do a quick assert on the length of both input dicts
assert len(CAPACITY) == len(C)

First we create our gurobi model:

In [10]:
# Creates the prob variable to contain the problem data
m = gp.Model('TransShipment')

Restricted license - for non-production use only - expires 2024-10-28


And finally, we can declare our decision variables: for each arc we define the flow. By setting the data-type with vtype as an Integer we restrict the decision variables to be 0 or higher.

In [14]:
x = m.addVars(C, vtype=GRB.INTEGER, name='x')
x

{('1', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('1', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('2', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('4', '5'): <gurobi.Var *Awaiting Model Update*>}

## Objective and constraints
The objective function which we want to minimize is the cost of the flow in the network.
Thus, the sum over the cost of each arc multiplied by the flow variable for the arc.

In [15]:
#Set the objective function as the product of the flow number of each arch multipled by the cost of that arc.
m.setObjective(x.prod(C), GRB.MINIMIZE)

Next we specify our flow conservation constraint in the network. For each node i in the system, we constrain the difference between the incoming flow and outgoing flow to be equal to the specified supply for that node.

In [17]:
# Add flow conservation constraints
for i in N:
  m.addConstr(x.sum(i,'*')-x.sum('*',i) == D[i], name=f"ConservationOfFlow{i}")

To constrain each arc at its capacity, we iterate through all the decision variables and add a constraint for each.

In [19]:
# Add capacity constraints
for (o, d), var in x.items():
  m.addConstr(var <= CAPACITY[(o,d)], name=f"CapacityConstraints_{o}_{d}")

In [20]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 14 columns and 21 nonzeros
Model fingerprint: 0xfabf43ce
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 53.0000000
Presolve removed 13 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 47 53 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.700000000000e+01, best bound 4.700000000000e+01, gap 0.0000%


In [21]:
print(f"Optimal objective value: {m.objVal}")

print("\nShipment plan:")
for o, d in x.keys():
    if (abs(x[o, d].x) > 0): #Only print if not 0
        print (f"Ship {x[o,d].x} units from node {o} to node {d}")

Optimal objective value: 47.0

Shipment plan:
Ship 3.0 units from node 1 to node 2
Ship 2.0 units from node 1 to node 3
Ship 5.0 units from node 2 to node 3
Ship 5.0 units from node 3 to node 4
Ship 1.0 units from node 3 to node 5
Ship 1.0 units from node 3 to node 6
Ship 3.0 units from node 4 to node 5


Our mimimal transport cost is 47 and we have the flow for each arc given by the decision variables.

![slide2.png](https://i.ibb.co/wWYbzJw/download-1.png)

## Second Transhipment point

We have the opportunity to place a second transshipment point that is more centrally located. However there is a much higher fixed cost attached for using this point. Are the lower transport costs worth this?

In [ ]:
#New transport cost OD matrix, with added transshipment point 7.
#Lower transport costs to stores, higher from factories.
C = {('1', '2'): 1,
     ('1', '3'): 4,
     ('1', '7'): 5,
     ('2', '3'): 2,
     ('2', '7'): 3,
     ('3', '4'): 2,
     ('3', '5'): 5,
     ('3', '6'): 8,
     ('4', '5'): 1,
     ('7', '4'): 1,
     ('7', '5'): 1,
     ('7', '6'): 7}


N = [str(i+1) for i in range(7)]
N

supply = [5, 2, 0, -2, -4, -1, 0]

D = dict(zip(N, supply))
D

#New capacity OD matrix, with added transshipment point 7
CAPACITY = { ('1', '2'): 3,
             ('1', '3'): 3,
             ('1', '7'): 3,
             ('2', '3'): 7,
             ('2', '7'): 7,
             ('3', '4'): 5,
             ('3', '5'): 7,
             ('3', '6'): 1,
             ('4', '5'): 3,
             ('7', '4'): 5,
             ('7', '5'): 7,
             ('7', '6'): 1}

#Cost for using a node
node_costs = {'7' : 4, '3' : 1}

E = [(i,j) for i in N for j in N if i in C.keys() if j in C[i].keys()]

In [ ]:
# Define our new model
m =

In [ ]:
# Add decision variables
x =
x

Add a new binary variable that indicates whether we use transhippment node 3 or 7

In [ ]:
# Add new binary variable that indicates whether we use transshipment node 3 or 7
y =
y

In [ ]:
#Set the objective function as the product of the flow number of each arch multipled by the cost of that arc.
# Note that we now also added the unit costs of running either transhipment point 3 or 7


Next we will again specify our flow conservation constraint in the network. For each node i in the system, we constrain the difference between the incoming flow and outgoing flow to be equal to the specified supply for that node.

In [ ]:
# Add flow constraint
for i in N:


To again constrain each arc at its capacity, we iterate through all the decision variables and add a constraint for each.

In [ ]:
# Add capacity constraint
for (o, d), var in x.items():


In [ ]:
network_capacity = sum(CAPACITY.values())
#Adding linking constraints to limit the in and outgoing flow to 0, if that node is not being used.



In [ ]:
y

In [ ]:
# Optimize model


In [ ]:
print(f"Optimal objective value: {m.objVal}")

for n, var in y.items():
    if (abs(var.x) > 0): #Only print if not 0
        print (f"Use transhipment point {n}")
print ("Shipment plan")
for (o,d), var in x.items():
    if (abs(var.x) > 0): #Only print if not 0
        print (f"Ship {x[o,d].x} units from node {o} to node {d}")

Running this model indicates that a solution with only node 7 has the lowest transport costs.
This indicates that renting the space for more centrally located transhipment will pay itself back in savings on transport costs.